In [2]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import pandas as pd
import numpy as np
import time
#from tqdm import tqdm
#from scipy.sparse import csr_matrix, csc_matrix, lil_matrix
#from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#from itertools import combinations
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from surprise import Dataset, Reader
from surprise import SVD, accuracy
from surprise.model_selection import train_test_split as train_test_split_s

In [ ]:
activity_train = pd.read_csv("activity_train.csv", names=["protein", "molecule", "activity"])
activity_train["molecule"] = activity_train["molecule"].apply(lambda x: x.strip())
activity_test = pd.read_csv("activity_test_blanked.csv", names=["protein", "molecule", "activity"]).iloc[:,:2]
activity_test["molecule"] = activity_test["molecule"].apply(lambda x: x.strip())

In [3]:
mol_bits = pickle.load(open("mol_bits.pkl", "rb"))

mol_features = {}
for mol, feat in mol_bits.items():
    mol_features.setdefault(mol, torch.zeros(2048, dtype=torch.float32))
    for f in feat:
        mol_features[mol][f] = 1

In [4]:
combined_prot = set(np.concatenate([activity_train['protein'].unique(), activity_test['protein'].unique()]))
prot_mappings = {k: i for i, k in enumerate(combined_prot)}

In [5]:
combined_mol = set(np.concatenate([activity_train['molecule'].unique(), activity_test['molecule'].unique()]))
mol_mappings = {k: i for i, k in enumerate(combined_mol)}

## Neural Collaborative Filtering

##### https://doi.org/10.1145/3038912.3052569
He et al. 2017. Neural Collaborative Filtering. In Proceedings of the 26th International Conference on World Wide Web (WWW '17). International World Wide Web Conferences Steering Committee, Republic and Canton of Geneva, CHE, 173–182.

In [6]:
class NCF(nn.Module):
    def __init__(self, prot_vocab, mol_vocab, num_feats, embedding_dim, drop_prob=0.5):
        super().__init__()
        self.protein_embedding = nn.Embedding(prot_vocab, embedding_dim)
        self.molecule_embedding = nn.Embedding(mol_vocab, embedding_dim)
        
        self.fc1 = nn.Linear(2 * embedding_dim + num_feats, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 1)
        
        self.dropout = nn.Dropout(drop_prob)
        
        
    def forward(self, prot, mol, feat):
        prot_emb = self.protein_embedding(prot)
        mol_emb = self.molecule_embedding(mol)
        
        concat = torch.cat([prot_emb, mol_emb, feat], dim=-1)
        x = F.relu(self.fc1(concat))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [7]:
class InteractionDataset(Dataset):
    def __init__(self, interaction_df, prot_mappings, mol_mappings, mol_feats, test=False, pred=False):
        self.prot_mappings = prot_mappings
        self.mol_mappings = mol_mappings
        self.mol_feats = mol_feats
        self.interaction_df = interaction_df
        self.test = test
        self.pred = pred

    def __getitem__(self, index):
        inputs = self.interaction_df.iloc[index,:]
        prot_idx = self.prot_mappings[inputs[0]]
        mol_idx = self.mol_mappings[inputs[1]]
        mol_feats = self.mol_feats[inputs[1]]
        if self.test:
            return inputs[0], inputs[1], prot_idx, mol_idx, mol_feats, torch.tensor(inputs[2], dtype=torch.float32)
        if self.pred:
            return prot_idx, mol_idx, mol_feats
        return prot_idx, mol_idx, mol_feats, torch.tensor(inputs[2], dtype=torch.float32)
                                          
    def __len__(self):
        return len(self.interaction_df)

In [8]:
train_df, temp = train_test_split(activity_train, test_size=0.2, random_state=13, shuffle=True, stratify=activity_train['protein'])
val_df, test_df = train_test_split(temp, test_size=0.5, random_state=13, shuffle=True, stratify=temp['protein'])

In [9]:
TRAIN = False
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EPOCHS = 100
BATCH_SIZE = 128
EMBEDDING_DIM = 32
LR = 5e-5

num_prots = len(prot_mappings)
num_mols = len(mol_mappings)
num_features = 2048

train_dataset = InteractionDataset(train_df, prot_mappings, mol_mappings, mol_features)
val_dataset = InteractionDataset(val_df, prot_mappings, mol_mappings, mol_features)
test_dataset = InteractionDataset(test_df, prot_mappings, mol_mappings, mol_features, test=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset)

model = NCF(num_prots, num_mols, num_features, EMBEDDING_DIM).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS)
loss_fn = nn.MSELoss()

In [10]:
def train(model, train_loader, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    for prot, mol, feat, act in train_loader:
        prot = prot.to(DEVICE)
        mol = mol.to(DEVICE)
        feat = feat.to(DEVICE)
        act = act.to(DEVICE)
        
        optimizer.zero_grad()
        preds = model(prot, mol, feat)
        loss = loss_fn(preds, act)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    return running_loss / len(train_loader)

def evaluate(model, val_loader, loss_fn):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for prot, mol, feat, act in val_loader:
            prot = prot.to(DEVICE)
            mol = mol.to(DEVICE)
            feat = feat.to(DEVICE)
            act = act.to(DEVICE)
            
            preds = model(prot, mol, feat)
            loss = loss_fn(preds, act)
            
            running_loss += loss.item()
    
    return running_loss / len(val_loader)

In [11]:
def trainer(model, train_dataloader, val_dataloader, loss_fn, optimizer):
    train_losses = []
    val_losses = []
    for epoch in range(EPOCHS):
        train_loss = train(model, train_dataloader, loss_fn, optimizer)
        train_losses.append(train_loss)
        val_loss = evaluate(model, val_dataloader, loss_fn)
        val_losses.append(val_loss)
        scheduler.step()
        if epoch % 10 == 9:
            torch.save(model.state_dict(), f"model-epoch{epoch+1}.pth")
        print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    return train_losses, val_losses

In [12]:
if TRAIN:
    train_losses, val_losses = trainer(model, train_dataloader, val_dataloader, loss_fn, optimizer)

In [13]:
if TRAIN:
    plt.plot(range(1,EPOCHS+1), train_losses, label="Train")
    plt.plot(range(1,EPOCHS+1), val_losses, label="Val")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig("train_val_perf.png")
    plt.show()

In [14]:
summary(model)

Layer (type:depth-idx)                   Param #
NCF                                      --
├─Embedding: 1-1                         4,608
├─Embedding: 1-2                         2,363,680
├─Linear: 1-3                            2,163,712
├─Linear: 1-4                            262,400
├─Linear: 1-5                            16,448
├─Linear: 1-6                            65
├─Dropout: 1-7                           --
Total params: 4,810,913
Trainable params: 4,810,913
Non-trainable params: 0

#### Evaluation MLP

In [15]:
MODEL_PATH = "model-epoch100.pth"
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device(DEVICE)))

<All keys matched successfully>

In [16]:
test_set = {
    'protein': [],
    'molecule': [],
    'activity (truth)': [],
    'activity (prediction)': []
}

model.eval()
with torch.no_grad():
    for prot, mol, prot_id, mol_id, feat, act in test_dataloader:
        prot_id = prot_id.to(DEVICE)
        mol_id = mol_id.to(DEVICE)
        feat = feat.to(DEVICE)
            
        preds = model(prot_id, mol_id, feat)
        preds = preds.cpu().detach().item()
        
        test_set['protein'].append(prot[0])
        test_set['molecule'].append(mol[0])
        test_set['activity (truth)'].append(int(act.item()))
        test_set['activity (prediction)'].append(round(preds, 4))

In [17]:
rmse = np.sqrt(np.mean((np.array(test_set['activity (truth)']) - np.array(test_set['activity (prediction)'])) ** 2))
print(f"RMSE: {rmse:.4f}")

RMSE: 2.6739


In [18]:
pd.DataFrame(test_set).head(10)

,protein,molecule,activity (truth),activity (prediction)
0,Q14289,CHEMBL2007002,1,2.0026
1,P35462,CHEMBL1926700,8,5.5573
2,P20309,CHEMBL4544086,1,3.4996
3,P35368,CHEMBL440865,10,6.9973
4,O43614,CHEMBL3663477,3,3.3373
5,P50406,CHEMBL4093110,7,2.8840
6,P28223,CHEMBL77418,4,3.8005
7,P41145,CHEMBL2113297,5,4.4913
8,P05067,CHEMBL2069430,1,2.8991
9,P34998,CHEMBL268043,7,5.5152


#### Prediction MLP

In [19]:
pred_dataset = InteractionDataset(activity_test, prot_mappings, mol_mappings, mol_features, pred=True)
pred_dataloader = DataLoader(dataset=pred_dataset)

In [20]:
activity_preds_mlp = []
model.eval()
with torch.no_grad():
    for prot_id, mol_id, feat in pred_dataloader:
        prot_id = prot_id.to(DEVICE)
        mol_id = mol_id.to(DEVICE)
        feat = feat.to(DEVICE)
            
        pred = model(prot_id, mol_id, feat)
        pred = pred.cpu().detach().item()
        
        activity_preds_mlp.append(np.clip(pred, 1, 10))

In [21]:
preds_mlp = pd.concat([activity_test, pd.Series(activity_preds_mlp, name="activity")], axis=1)

In [22]:
preds_mlp.head(10)

,protein,molecule,activity
0,O14842,CHEMBL2022258,6.186503
1,O14842,CHEMBL2047161,6.518781
2,O14842,CHEMBL2047163,6.378317
3,O14842,CHEMBL2047168,5.591493
4,O14842,CHEMBL2047169,5.641742
5,O14842,CHEMBL2048621,6.223549
6,O14842,CHEMBL2048623,5.873072
7,O14842,CHEMBL207881,1.293392
8,O14842,CHEMBL4067052,2.126847
9,O14842,CHEMBL4069191,3.815628


## User-User Collaborative Filtering

In [23]:
def RowCenterMatrix(M):
    row_means = np.expand_dims(np.nanmean(M, axis=1), axis=1)
    MC = M - row_means
    MC[np.isnan(MC)] = 0
    return MC


def CosSim_Matrix(M):
    norms = np.sqrt(np.sum(np.square(M), axis=1))
    norms[norms < 1e-6] = 1e-6 #this will solve rows or cols without variance
    norms_M = np.outer(norms, norms)
    return np.dot(M, M.T) / norms_M


def make_GBAMatrix(M):
    global_mean = np.nanmean(M)
    row_means = np.nanmean(M, axis=1)
    prot_dif = row_means - global_mean
    col_means = np.nanmean(M, axis=0)
    mol_dif = col_means - global_mean
    return global_mean, prot_dif, mol_dif


def estimate_score(mat, SM, gba, nn, row, col, verbose=False):
    N,M = mat.shape
    sims = list(zip(SM[row], range(N)))
    sims.sort()
    sims.reverse()
    
    cnt, S, Ssims = 0, 0, 0
    if verbose: print("row: %d - col: %d" % (row,col))
    for  sim, idx in sims[1:]:
        if not np.isnan(mat[idx, col]) and sim > 0:
            r = gba[0] + gba[1][idx] + gba[2][col]
            S += sim * (mat[idx, col] - r)
            Ssims += sim
            cnt += 1
            if verbose: print("\tItem:%4d    Score:%4.1f (Sim: %6.3f)" %(idx, mat[idx, col], sim))
        if cnt >= nn: break
    
    r = gba[0] + (gba[1][row] if not np.isnan(gba[1][row]) else 0) + (gba[2][col] if not np.isnan(gba[2][col]) else 0)

    if Ssims <= 0:
        if verbose: print("\tNo similars: outputing the Global Baseline Average")
        if verbose: print("\tScore: %7.4f" % r)
        return r
    if verbose: print("\tScore: %7.4f" % (r + S / Ssims))

    return r + S / Ssims

In [24]:
cols = activity_train['molecule'].map(mol_mappings).values
rows = activity_train['protein'].map(prot_mappings).values
data = activity_train['activity'].values

uu_matrix = np.empty((len(prot_mappings), len(mol_mappings)))
uu_matrix[:] = np.nan
uu_matrix[rows, cols] = data

In [25]:
gba = make_GBAMatrix(uu_matrix)
gba[0], gba[1].shape, gba[2].shape

(4.708792949724046, (144,), (73865,))

In [26]:
uu_c_matrix = RowCenterMatrix(uu_matrix)
uu_sim_matrix = CosSim_Matrix(uu_c_matrix)

#### Evaluation U-U CF

In [27]:
testing = []
for i, (prot, mol, _) in activity_train.iterrows():
    prot_idx = prot_mappings[prot]
    mol_idx = mol_mappings[mol]
    pred = estimate_score(uu_matrix, uu_sim_matrix, gba, 10, prot_idx, mol_idx)
    testing.append(np.clip(pred, 1, 10))

In [28]:
testing_df = pd.concat([activity_train, pd.Series(testing, name="activity (pred)")], axis=1)

In [29]:
print(f"RMSE: {np.sqrt(np.mean((testing_df['activity'] - testing_df['activity (pred)']) ** 2)):.4f}")

RMSE: 2.2150


In [30]:
testing_df.head(10)

,protein,molecule,activity,activity (pred)
0,O14842,CHEMBL2022243,4,4.662175
1,O14842,CHEMBL2022244,6,6.662175
2,O14842,CHEMBL2022245,2,2.662175
3,O14842,CHEMBL2022246,1,1.662175
4,O14842,CHEMBL2022247,4,4.662175
5,O14842,CHEMBL2022248,4,4.662175
6,O14842,CHEMBL2022249,3,3.662175
7,O14842,CHEMBL2022250,1,1.662175
8,O14842,CHEMBL2022251,1,1.662175
9,O14842,CHEMBL2022252,6,6.662175


#### Prediction U-U CF

In [31]:
activity_preds_cf = []
for i, (prot, mol) in activity_test.iterrows():
    prot_idx = prot_mappings[prot]
    mol_idx = mol_mappings[mol]
    pred = estimate_score(uu_matrix, uu_sim_matrix, gba, 10, prot_idx, mol_idx)
    activity_preds_cf.append(np.clip(pred, 1, 10))

In [32]:
preds_uu_cf = pd.concat([activity_test, pd.Series(activity_preds_cf, name="activity")], axis=1)
preds_uu_cf.head(10)

,protein,molecule,activity
0,O14842,CHEMBL2022258,5.370968
1,O14842,CHEMBL2047161,5.370968
2,O14842,CHEMBL2047163,5.370968
3,O14842,CHEMBL2047168,5.370968
4,O14842,CHEMBL2047169,5.370968
5,O14842,CHEMBL2048621,5.370968
6,O14842,CHEMBL2048623,5.370968
7,O14842,CHEMBL207881,3.328841
8,O14842,CHEMBL4067052,5.370968
9,O14842,CHEMBL4069191,5.370968


---

## SVD Matrix Factorization

In [33]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(activity_train[['protein', 'molecule', 'activity']], reader)

train, val = train_test_split_s(data, test_size=0.2, random_state=13, shuffle=True)

model = SVD(n_factors=32, random_state=13)
model.fit(train)
preds = model.test(val)
rmse = accuracy.rmse(preds)

RMSE: 2.4395


#### Prediction SVD MF

In [34]:
preds_svd_mf = activity_test.copy()
preds_svd_mf['activity'] = 0

In [35]:
test_data = Dataset.load_from_df(preds_svd_mf[['protein', 'molecule', 'activity']], reader)
test = test_data.build_full_trainset().build_testset()

In [36]:
preds = model.test(test)
preds_list = [np.clip(pred.est, 1, 10) for pred in preds]
preds_svd_mf['activity'] = preds_list
preds_svd_mf.head(10)

,protein,molecule,activity
0,O14842,CHEMBL2022258,5.331880
1,O14842,CHEMBL2047161,5.331880
2,O14842,CHEMBL2047163,5.331880
3,O14842,CHEMBL2047168,5.331880
4,O14842,CHEMBL2047169,5.331880
5,O14842,CHEMBL2048621,5.331880
6,O14842,CHEMBL2048623,5.331880
7,O14842,CHEMBL207881,5.342925
8,O14842,CHEMBL4067052,5.331880
9,O14842,CHEMBL4069191,5.331880


---

## Ensemble

In [37]:
preds_ensemble = activity_test.copy()
preds_ensemble['activity'] = (preds_uu_cf['activity'] + preds_mlp['activity'] + preds_svd_mf['activity']) / 3

In [38]:
preds_ensemble.head(10)

,protein,molecule,activity
0,O14842,CHEMBL2022258,5.629784
1,O14842,CHEMBL2047161,5.740543
2,O14842,CHEMBL2047163,5.693722
3,O14842,CHEMBL2047168,5.431447
4,O14842,CHEMBL2047169,5.448197
5,O14842,CHEMBL2048621,5.642132
6,O14842,CHEMBL2048623,5.525306
7,O14842,CHEMBL207881,3.321719
8,O14842,CHEMBL4067052,4.276565
9,O14842,CHEMBL4069191,4.839492


#### Saving Predictions CSV

In [39]:
preds_ensemble.to_csv("preds_06.txt", index=False, header=False)